In [24]:
import numpy as np
#模拟数据
m=26
n=m
import time
inputDim=[n,m]    #每层交换机个数,3:最高层个数，6：最底层个数；因为我们的推导是从顶层向底层推，所以我把array倒过来
D=[3*np.random.rand(inputDim[i+1]) for i in range(len(inputDim)-1)] #交换机个数为4、5、6的层的单个交换机延时

#structure=[[np.random.randint(0, inputDim[i]-1) for _ in range(inputDim[i+1])] for i in range(len(inputDim)-1)]  
structure=list(range(n))
np.random.shuffle(structure)
structure=[structure]
#structure: [[1, 2, 1, 1], [0, 3, 3, 3, 0], [1, 0, 4, 0, 3, 0]]  
#其中[1, 2, 1, 1]指的是交换机个数为4的层与交换机个数为3的层之间的交换机连接情况；其中的第i个数a[i]表示交换机个数为4的层中第i-1台交换机与换机个数为3的层中第a[i]台交换机相连
T=[2*np.zeros([inputDim[i],inputDim[i]]) for i in range(len(inputDim))]#各层的同级交换机间通信时间, 最开始展现的是PSW，最后print的是ASW
T[0]=2*np.random.rand(inputDim[0],inputDim[0])
for k in range(1,len(inputDim)):
    for i in range(inputDim[k]):
        for j in range(inputDim[k]):
            T[k][i][j]=D[k-1][i]+D[k-1][j]+T[k-1][structure[k-1][i]][structure[k-1][j]]
#临时矩阵
TTT=np.zeros([m,m,n,n])
for i in range(m):
    for j in range(m):
        d1=np.ones([n,n])*D[-1][i]
        d2=np.ones([n,n])*D[-1][j]
        t0=np.ones([n,n])*T[-1][i][j]
        TTT[i][j]=abs(np.array(T[0])+d1+d2-t0)

#T'矩阵
TT=np.zeros([m*n,m*n])
for i in range(m):
    for j in range(m):
        for i1,k in enumerate(range((i*n),((i+1)*n),1)):
            for i2,l in enumerate(range((j*n),((j+1)*n),1)):
                TT[k][l]=TTT[i][j][i1][i2]
                
L=np.zeros(m*n)
for i in range(m):
    L[(i*n)+structure[-1][i]]=1
L2=np.zeros(m*n)
for i in range(m):
    L2[structure[-1][i]*m+i]=1


In [25]:
def target(y):    #target of argmin
    return y.T.dot(TT).dot(y)+(QList+gList).T.dot(g(y))+alpha*(np.linalg.norm(y-x))**2   
bnds = [(0, 1)]*(m*n)  # 定义域
alpha=1
max_iter=10000
eps=4e-3   #可能要调，影响精度
def g1(x,i,j):
    return x[i*n+j]*(x[i*n+j]-1)
def g2(x,i):
    return sum(x[(i*n):(i*n+n)])-1 
def g(x):
    res=[]
    for i in range(m):
        for j in range(n):
            res.append(g1(x,i,j))
    for i in range(m):
        res.append(g2(x,i))
    for i in range(m):
        for j in range(n):
            res.append(-g1(x,i,j))
    for i in range(m):
        res.append(-g2(x,i))
    return np.array(res)


x,x_bar=np.random.random([m*n]),np.zeros([m*n])
x_new=x.copy()
gList=g(x)
QList=[max(0,-i) for i in gList]#[max(0,-i) for i in gList]
ycurrent=np.zeros(m*n)


start=time.time()
for t in range(max_iter):
    if t%2==0:
        print(t)

    #构造和要argmin的函数等价的二次项和一次项系数：new_TT和linear
    dia=np.array(QList[:m*n])+np.array(gList[:m*n])-(np.array(QList)[(m*n+m):(2*m*n+m)]+np.array(gList)[(m*n+m):(2*m*n+m)])+alpha*np.ones(m*n)
    new_TT=TT+np.diag(dia)
    epsilon= np.linalg.norm(1/np.linalg.eigvals(new_TT)[0])/1.03     #可能要调，影响迭代次数和精度
    #print('---',epsilon)
    
    tmp1=np.array(QList)[(m*n):(m*n+m)]+np.array(gList)[(m*n):(m*n+m)]
    tmp2=np.array([[i]*n for i in tmp1]).reshape(m*n)
    tmp3=np.array(QList)[-m:]+np.array(gList)[-m:]
    tmp4=np.array([[i]*n for i in tmp3]).reshape(m*n)
    linear=-(np.array(QList)[:m*n]+np.array(gList)[:m*n])+tmp2+np.array(QList)[(m*n+m):(2*m*n+m)]+np.array(gList)[(m*n+m):(2*m*n+m)]-tmp4-2*alpha*x


    #求解目标是x_k1
    x_k=x.copy()
    x_k1=np.clip(x-epsilon*(2*new_TT.dot(x)+linear),0,1)
    while np.linalg.norm(x_k-x_k1)>eps:
        x_k=x_k1.copy()
        x_k1=np.clip(x_k-epsilon*(2*new_TT.dot(x_k)+linear),0,1)
        print(np.linalg.norm(x_k-x_k1))


    x_new=x_k1.copy()
    if np.linalg.norm(x-x_new)<eps:
        break
    x=x_new.copy()
    gList=g(x)
    QList=[max(-gList[i],QList[i]+gList[i]) for i in range(len(gList))]
    x_bar=x_bar*t/(t+1)+x/(t+1)
    
print('time',time.time()-start)
#发现：new_TT主对角线元素整体尽可能小，则收敛更快；学习率epsilon尽可能小，则精度越高且速度并不很慢（我没做几轮实验，这个说法不大科学）
print(sum(abs(x-L)))
xx=x.reshape([m,n])
a=np.array([np.argmax(xx[i]) for i in range(m)])
print(a-structure)   #理想情况应该每个元素都是0

0
0.08105050473462878
0.062152903779057196
0.047533744922371036
0.038731393252539964
0.03337545596985419
0.02203982331213497
0.020968121451703677
0.013019106357941691
0.011542326817802179
0.011364612781057945
0.01118963495854109
0.0008961666113946522
0.6771822510990978
0.5892607948246329
0.49709764659562417
0.41821546379824764
0.33186109034985106
0.2633628726444543
0.18050390808568767
0.13049298406425316
0.06604425079679843
0.050578717001625084
0.04370104564431321
0.042301550090823245
0.041392676340223596
0.04043461739543739
0.03850597665622831
0.037833819536646385
0.03719320801459153
0.036278107532158474
0.035742600154887096
0.0356366527080818
0.035904543925344694
0.03456611159864547
0.03413097406565788
0.03309484785726864
0.03278191596641349
0.032773486629956244
0.03170478390619351
0.030508022562584686
0.030370103390719307
0.029402952317442076
0.029034424470273674
0.0286332197033282
0.028048747943591978
0.02758631826492287
0.026304037538327944
0.026104698482017116
0.02581122605409749

0.06342209176480615
0.0656355954741056
0.06794133538995299
0.06854840940732225
0.07077503558959948
0.07317626766940102
0.075688816758566
0.07662609342926086
0.07894794393062692
0.08079088273140926
0.0820242494177257
0.08262329100284925
0.083368884109262
0.08553061729663541
0.08553305697895676
0.08716033435910805
0.08928524163383011
0.09150820472320921
0.09232664724494807
0.0933010434315765
0.09394646185110224
0.09285627999993835
0.09321505584118088
0.09288095907629393
0.09403011178911877
0.0950626089407476
0.09351871020428996
0.09439752140501462
0.09585777763351976
0.09413210074673142
0.09047789947545035
0.08907955033215449
0.08917571848430203
0.08905361664310817
0.08873962008019881
0.08757101655176269
0.0866416681913698
0.08474863533303739
0.08206914564702794
0.07881526452992137
0.07775691927816303
0.07523261199914301
0.07052496745684632
0.06532162941478731
0.05802095028118377
0.05259143550640111
0.04721890908173892
0.04696251620643286
0.04680436140457507
0.04664709799715062
0.0464907

In [27]:
gList=g(L)
QList=[max(-gList[i],QList[i]+gList[i]) for i in range(len(gList))]
np.array(QList[:m*n])+np.array(gList[:m*n])-(np.array(QList)[(m*n+m):(2*m*n+m)]+np.array(gList)[(m*n+m):(2*m*n+m)])

array([0.2483231 , 0.09695472, 0.19569076, 0.24783574, 0.2375196 ,
       0.24737704, 0.04733061, 0.22686741, 0.06781159, 0.12128465,
       0.2132154 , 0.15841318, 0.17020174, 0.18819892, 0.23060659,
       0.16314331, 0.21299846, 0.09980454, 0.24181603, 0.23009604,
       0.23100719, 0.24569057, 0.23901373, 0.20390144, 0.13527712,
       0.23693241, 0.21011063, 0.14788818, 0.01786728, 0.2435212 ,
       0.23912883, 0.22100071, 0.08978673, 0.22670013, 0.22232426,
       0.10905326, 0.13912599, 0.06423631, 0.1323046 , 0.22138525,
       0.05713572, 0.24926718, 0.10883347, 0.23520823, 0.10302569,
       0.10170875, 0.04830934, 0.09328926, 0.17642522, 0.18337559,
       0.21527713, 0.03546221, 0.24994596, 0.2157887 , 0.10050829,
       0.17867819, 0.18268036, 0.17323943, 0.21671026, 0.24404743,
       0.21442919, 0.15421953, 0.20494348, 0.16853243, 0.2029476 ,
       0.0825656 , 0.24787348, 0.19638637, 0.24818439, 0.10730258,
       0.01226953, 0.18240101, 0.01818478, 0.24240026, 0.24925

In [22]:
np.diag(TT)

array([1.69525372e-01, 8.11501790e-01, 1.08142197e+00, 0.00000000e+00,
       6.63723441e-01, 6.01867624e-01, 1.43383086e+00, 1.16462703e-01,
       2.50537824e+00, 7.30206553e-01, 1.41231748e+00, 9.22987996e-01,
       1.93818648e+00, 9.31599345e-01, 1.62709936e+00, 8.89328044e-01,
       1.93924355e+00, 2.56711555e+00, 2.69001933e+00, 1.01279732e+00,
       1.11349239e+00, 2.12331688e+00, 8.32244876e-01, 1.64464103e+00,
       1.48042198e+00, 3.47343862e-01, 2.10771186e+00, 1.12668470e+00,
       8.56764517e-01, 1.93818648e+00, 2.60190993e+00, 1.33631886e+00,
       5.04355622e-01, 2.05464919e+00, 5.67191756e-01, 1.20797993e+00,
       5.25869008e-01, 2.86117448e+00, 0.00000000e+00, 2.86978583e+00,
       3.11087127e-01, 2.82751453e+00, 1.05706826e-03, 6.28929067e-01,
       7.51832844e-01, 9.25389164e-01, 3.05167888e+00, 1.85130400e-01,
       1.10594161e+00, 2.93545454e-01, 4.57764509e-01, 2.28553035e+00,
       1.81416640e+00, 8.33139241e-01, 5.63219063e-01, 1.64464103e+00,
      